<h1> WAvelet Based Broad LEarning System: WABBLES </h1>

WABBLES model implemented by Miguel Jara-Maldonado

Mapping nodes use a continuous wavelet function and the enhancement nodes use the sigmoid or tanh function.

The parameters of the model can be saved during the training process, or they can be loaded to avoid the training process or to train from a previously functional and trained model.

<h3> <font color="blue"> Import libraries and configure settings </font> </h3>

In [10]:
### ---- Libraries used by BLS 
import time
import random
import numpy as np
import sys
from scipy.stats import zscore

from WABBLES.processing.replaceNan import replaceNan
from WABBLES.processing.binarize_label import binarize_label
from WABBLES.processing.convert_x_y_to_array import convert_x_y_to_array
from WABBLES.processing.percentage import percentage
from WABBLES.processing.update_min_max import update_min_max
from WABBLES.processing.LightCurve import LightCurve
from WABBLES.processing.LightCurve import Set_Av_Class
from WABBLES.processing.color import color
from WABBLES.model.WABBLES import wabbles
### ----

import numpy.matlib
import math
import matplotlib.pyplot as plt
import glob # Reads the files
import csv  # Creates csv files
import pandas as pd #To generate data structures
from sklearn.model_selection import train_test_split # For cross-validation

import os


# <font color="blue">  Configurable variables </font> 

To operate this code, it is only necessary to modify the following variables.

In [2]:
# The parameters are loaded from a file instead of initialized
load_parameters = False

# The model is trained (either using the loaded parameters or the initialized parameters) 
train_model = True 

# Save the model parameters in a file? (It will automatically save each epoch that it obtains an better result)
save_parameters = True #It is recommended to use 1 run when train the model and saving the parameters

# Set this vairable to the path where you saved the light curves (LCs)
lc_path = "D:/Documentos/UDLAP/Tesis/Astronomía/Datasets/15000_datapoints/"

# Set the type of dataset that you want to use:
#  0 = The 3-median dataset
#  1 = The Real-LC dataset
dataset_simulation_type = 0


"""
 This function initializes all the variables that can be configured to optimize the model
 seed - used for generating random numbers
 num_class - number of classes
 runs - times that the model will be cross-validated (this is set to 1 because the cross-validation process occurs outside of BLS) 
 class_threshold - is the classification threshold, it can be chosen from 0 to 1, having a value closer to 0 means more priority to class 1 and having a value closer to 1 will prioritize class 0
 
 N1* - the number of mapped feature nodes
 N3* - the number of enhancement nodes
 
 err_threshold - Gradient descent error limit
 gd_epochs_options - limit of epochs to reduce the gradient descent error
 alpha_options - learning rate that decides how much the parameters are updated in each iteration
 
 wavelet_names - The wavelet to be used: morlet (morlet wavelet), gausD (Derivative of Gaussian DoG wavelet), mexHat (Mexican hat wavelet)
 phi_names - The phi function options: sigmoid, tanh (Hyperbolic tangencial)
 invert_LC - decides if the LC has been turned upsidedown or not

"""
def set_variables():
    seed = 1; 
    num_class = 2; 
    runs = 2; 
    class_threshold = 0.5

    N1_bls_options = [3] #10
    N3_bls_options = [4] # 100
    
    err_threshold = 0.005
    gd_epochs_options = [5] #20    
    alpha_options = [0.001] # .0001
    
    wavelet_names = ['mexHat']
    phi_names = ['tanh']    
    invert_LC = False # When true, the flux of the LC will be multiplied by -1, inverting the geometry of the LC
    
    return seed, num_class, runs, N1_bls_options, N3_bls_options, err_threshold, gd_epochs_options, alpha_options, wavelet_names, phi_names, class_threshold, invert_LC
  

<h3> <font color="blue"> Initializations </font> </h3>

In [3]:
#to get the current working directory
current_dir = os.getcwd() 
current_dir = current_dir.replace("\\", "/")

parameters_path = current_dir + "/parameters/" 

# Set this variable to the path where the csv files are going to be save to
csv_path = current_dir + "/results/" 

if dataset_simulation_type > 1:
    dataset_simulation_type = 0

if( dataset_simulation_type == 0 ):
    parameters_path = parameters_path + "3median_"
    #Path of the 3median dataset
    lc_path = lc_path + "Simulated 3 Median Dataset/2048_mean_binned/"
else:
    parameters_path = parameters_path + "realLC_"
    #Path of the realLC dataset
    lc_path = lc_path + "Real LCs with simulation transits/2048_mean_binned/"
    
max_rand = 4294967295 #Highest possible random number (for cross-validation)

seed, num_class, runs, N1_bls_options, N3_bls_options, err_threshold, gd_epochs_options, alpha_options, wavelet_names, phi_names, class_threshold, invert_LC = set_variables()

# <font color="blue">  Obtain the LCs from a file </font> 

In [9]:
light_Curves = list() #LCs are stored on this list
files = glob.glob(lc_path +'*_*.txt')  #Obtain all the LC file names

for slc_index in range(len(files)):
    
    file = open(lc_path + "binned_LC_" + str(slc_index) + ".txt","r")
    file_lines = file.readlines()

    line_index = 0

    for line in file_lines:
        if(line[0] == "#" or line[0] == "t"): #Ignore comments and labels
            continue

        if(line_index == 0): #Extract t0, t_duration, t_period, etc..
            parameters = line.split(",")
            t0 = float(parameters[0])
            t_duration = float(parameters[1])
            t_period = float(parameters[2])
            av_class = int(parameters[3])
            planet_name = parameters[4]
            limb_model = parameters[5]
            phi = float(parameters[6])
            wave_amp = float(parameters[7])
            wave_perd = float(parameters[8])
            PA = float(parameters[9])
            PW = float(parameters[10])
            sig_tol = float(parameters[11])

            light_Curves.append(LightCurve(planet_name, t0, t_duration, t_period, av_class, limb_model, slc_index))

            light_Curves[slc_index].phi = float(phi)
            light_Curves[slc_index].wave_amp = float(wave_amp)
            light_Curves[slc_index].wave_perd = float(wave_perd)
            light_Curves[slc_index].PA = float(PA)
            light_Curves[slc_index].PW = float(PW)
            light_Curves[slc_index].sig_tol = float(sig_tol)

        if(line_index > 0): #Extract the rest of the time and flux.
            #Extract flux and time values
            parameters = line.split(",")
            light_Curves[slc_index].time.append(float(parameters[0])) 
            
            #Inverting the LC will result in a 'n' shape instead of the 'u' shape, which may be beneficial for certain wavelets (for a better correlation)
            if invert_LC:
                flux = float(parameters[1].replace("\n", "")) * -1
            else:
                flux = parameters[1].replace("\n", "")
            
            light_Curves[slc_index].flux.append(float(flux)) 

        line_index += 1
    
print("Done!")

LC added from HATS-6 b added with class 0 and a limb model uniform. 0
LC added from HATS-6 b added with class 0 and a limb model linear. 1
LC added from HATS-6 b added with class 0 and a limb model quadratic. 2
LC added from HATS-6 b added with class 0 and a limb model nonlinear. 3
LC added from WASP-80 b added with class 0 and a limb model uniform. 4
LC added from WASP-80 b added with class 0 and a limb model linear. 5
LC added from WASP-80 b added with class 0 and a limb model quadratic. 6
LC added from WASP-80 b added with class 0 and a limb model nonlinear. 7
LC added from HAT-P-12 b added with class 0 and a limb model uniform. 8
LC added from HAT-P-12 b added with class 0 and a limb model linear. 9
LC added from HAT-P-12 b added with class 0 and a limb model quadratic. 10
LC added from HAT-P-12 b added with class 0 and a limb model nonlinear. 11
LC added from WASP-84 b added with class 0 and a limb model uniform. 12
LC added from WASP-84 b added with class 0 and a limb model linea

LC added from WASP-164 b added with class 0 and a limb model nonlinear. 119
LC added from WASP-64 b added with class 0 and a limb model uniform. 120
LC added from WASP-64 b added with class 0 and a limb model linear. 121
LC added from WASP-64 b added with class 0 and a limb model quadratic. 122
LC added from WASP-64 b added with class 0 and a limb model nonlinear. 123
LC added from WASP-91 b added with class 0 and a limb model uniform. 124
LC added from WASP-91 b added with class 0 and a limb model linear. 125
LC added from WASP-91 b added with class 0 and a limb model quadratic. 126
LC added from WASP-91 b added with class 0 and a limb model nonlinear. 127
LC added from WASP-89 b added with class 0 and a limb model uniform. 128
LC added from WASP-89 b added with class 0 and a limb model linear. 129
LC added from WASP-89 b added with class 0 and a limb model quadratic. 130
LC added from WASP-89 b added with class 0 and a limb model nonlinear. 131
LC added from WASP-104 b added with cla

LC added from WASP-175 b added with class 0 and a limb model uniform. 236
LC added from WASP-175 b added with class 0 and a limb model linear. 237
LC added from WASP-175 b added with class 0 and a limb model quadratic. 238
LC added from WASP-175 b added with class 0 and a limb model nonlinear. 239
LC added from WASP-83 b added with class 0 and a limb model uniform. 240
LC added from WASP-83 b added with class 0 and a limb model linear. 241
LC added from WASP-83 b added with class 0 and a limb model quadratic. 242
LC added from WASP-83 b added with class 0 and a limb model nonlinear. 243
LC added from HAT-P-39 b added with class 0 and a limb model uniform. 244
LC added from HAT-P-39 b added with class 0 and a limb model linear. 245
LC added from HAT-P-39 b added with class 0 and a limb model quadratic. 246
LC added from HAT-P-39 b added with class 0 and a limb model nonlinear. 247
LC added from KELT-21 b added with class 0 and a limb model uniform. 248
LC added from KELT-21 b added with

LC added from WASP-120 b added with class 0 and a limb model quadratic. 354
LC added from WASP-120 b added with class 0 and a limb model nonlinear. 355
LC added from CoRoT-8 b added with class 0 and a limb model uniform. 356
LC added from CoRoT-8 b added with class 0 and a limb model linear. 357
LC added from CoRoT-8 b added with class 0 and a limb model quadratic. 358
LC added from CoRoT-8 b added with class 0 and a limb model nonlinear. 359
LC added from WASP-147 b added with class 0 and a limb model uniform. 360
LC added from WASP-147 b added with class 0 and a limb model linear. 361
LC added from WASP-147 b added with class 0 and a limb model quadratic. 362
LC added from WASP-147 b added with class 0 and a limb model nonlinear. 363
LC added from WASP-158 b added with class 0 and a limb model uniform. 364
LC added from WASP-158 b added with class 0 and a limb model linear. 365
LC added from WASP-158 b added with class 0 and a limb model quadratic. 366
LC added from WASP-158 b added 

LC added from K2-285 b added with class 0 and a limb model quadratic. 474
LC added from K2-285 b added with class 0 and a limb model nonlinear. 475
LC added from TOI 270 b added with class 0 and a limb model uniform. 476
LC added from TOI 270 b added with class 0 and a limb model linear. 477
LC added from TOI 270 b added with class 0 and a limb model quadratic. 478
LC added from TOI 270 b added with class 0 and a limb model nonlinear. 479
LC added from TOI 125 c added with class 0 and a limb model uniform. 480
LC added from TOI 125 c added with class 0 and a limb model linear. 481
LC added from TOI 125 c added with class 0 and a limb model quadratic. 482
LC added from TOI 125 c added with class 0 and a limb model nonlinear. 483
LC added from Kepler-11 d added with class 0 and a limb model uniform. 484
LC added from Kepler-11 d added with class 0 and a limb model linear. 485
LC added from Kepler-11 d added with class 0 and a limb model quadratic. 486
LC added from Kepler-11 d added with

...

LC added from TOI 270 b added with class 0 and a limb model nonlinear. 9439
LC added from TOI 125 c added with class 0 and a limb model uniform. 9440
LC added from TOI 125 c added with class 0 and a limb model linear. 9441
LC added from TOI 125 c added with class 0 and a limb model quadratic. 9442
LC added from TOI 125 c added with class 0 and a limb model nonlinear. 9443
LC added from Kepler-11 d added with class 0 and a limb model uniform. 9444
LC added from Kepler-11 d added with class 0 and a limb model linear. 9445
LC added from Kepler-11 d added with class 0 and a limb model quadratic. 9446
LC added from Kepler-11 d added with class 0 and a limb model nonlinear. 9447
LC added from TOI 125 b added with class 0 and a limb model uniform. 9448
LC added from TOI 125 b added with class 0 and a limb model linear. 9449
LC added from TOI 125 b added with class 0 and a limb model quadratic. 9450
LC added from TOI 125 b added with class 0 and a limb model nonlinear. 9451
LC added from Keple

LC added from Empty Planet 42 added with class 2 and a limb model None. 9562
LC added from Empty Planet 43 added with class 2 and a limb model None. 9563
LC added from Empty Planet 44 added with class 2 and a limb model None. 9564
LC added from Empty Planet 45 added with class 2 and a limb model None. 9565
LC added from Empty Planet 46 added with class 2 and a limb model None. 9566
LC added from Empty Planet 47 added with class 2 and a limb model None. 9567
LC added from Empty Planet 48 added with class 2 and a limb model None. 9568
LC added from Empty Planet 49 added with class 2 and a limb model None. 9569
LC added from Empty Planet 50 added with class 2 and a limb model None. 9570
LC added from Empty Planet 51 added with class 2 and a limb model None. 9571
LC added from Empty Planet 52 added with class 2 and a limb model None. 9572
LC added from Empty Planet 53 added with class 2 and a limb model None. 9573
LC added from Empty Planet 54 added with class 2 and a limb model None. 9574

LC added from Empty Planet 163 added with class 2 and a limb model None. 9683
LC added from Empty Planet 164 added with class 2 and a limb model None. 9684
LC added from Empty Planet 165 added with class 2 and a limb model None. 9685
LC added from Empty Planet 166 added with class 2 and a limb model None. 9686
LC added from Empty Planet 167 added with class 2 and a limb model None. 9687
LC added from Empty Planet 168 added with class 2 and a limb model None. 9688
LC added from Empty Planet 169 added with class 2 and a limb model None. 9689
LC added from Empty Planet 170 added with class 2 and a limb model None. 9690
LC added from Empty Planet 171 added with class 2 and a limb model None. 9691
LC added from Empty Planet 172 added with class 2 and a limb model None. 9692
LC added from Empty Planet 173 added with class 2 and a limb model None. 9693
LC added from Empty Planet 174 added with class 2 and a limb model None. 9694
LC added from Empty Planet 175 added with class 2 and a limb mod

LC added from Empty Planet 275 added with class 2 and a limb model None. 9795
LC added from Empty Planet 276 added with class 2 and a limb model None. 9796
LC added from Empty Planet 277 added with class 2 and a limb model None. 9797
LC added from Empty Planet 278 added with class 2 and a limb model None. 9798
LC added from Empty Planet 279 added with class 2 and a limb model None. 9799
LC added from Empty Planet 280 added with class 2 and a limb model None. 9800
LC added from Empty Planet 281 added with class 2 and a limb model None. 9801
LC added from Empty Planet 282 added with class 2 and a limb model None. 9802
LC added from Empty Planet 283 added with class 2 and a limb model None. 9803
LC added from Empty Planet 284 added with class 2 and a limb model None. 9804
LC added from Empty Planet 285 added with class 2 and a limb model None. 9805
LC added from Empty Planet 286 added with class 2 and a limb model None. 9806
LC added from Empty Planet 287 added with class 2 and a limb mod

LC added from Empty Planet 392 added with class 2 and a limb model None. 9912
LC added from Empty Planet 393 added with class 2 and a limb model None. 9913
LC added from Empty Planet 394 added with class 2 and a limb model None. 9914
LC added from Empty Planet 395 added with class 2 and a limb model None. 9915
LC added from Empty Planet 396 added with class 2 and a limb model None. 9916
LC added from Empty Planet 397 added with class 2 and a limb model None. 9917
LC added from Empty Planet 398 added with class 2 and a limb model None. 9918
LC added from Empty Planet 399 added with class 2 and a limb model None. 9919
LC added from Empty Planet 400 added with class 2 and a limb model None. 9920
LC added from Empty Planet 401 added with class 2 and a limb model None. 9921
LC added from Empty Planet 402 added with class 2 and a limb model None. 9922
LC added from Empty Planet 403 added with class 2 and a limb model None. 9923
LC added from Empty Planet 404 added with class 2 and a limb mod

# <font color="blue">  Plots </font> 
Uncomment this section to plot the data

In [10]:
"""for i_lc in range(len(light_Curves)):
    fig = plt.figure()
    plt.plot(light_Curves[i_lc].flux)
    plt.xlabel('Time')
    plt.ylabel('Flux')
    plt.title(light_Curves[i_lc].av_class)"""


"for i_lc in range(5000, 5050):\n    fig = plt.figure()\n    plt.plot(light_Curves[i_lc].flux)\n    plt.xlabel('Time')\n    plt.ylabel('Flux')\n    plt.title(light_Curves[i_lc].av_class)"

# <font color="blue">  Prepare the datasets </font> 


In [11]:
light_curves = list()
light_curve_labels = list()

# Create a vector for the LCs and another for the labels
for i_lc in range(len(light_Curves)):
    light_curves.append(light_Curves[i_lc].flux.copy())
    light_curve_labels.append(binarize_label(light_Curves[i_lc].av_class))

# <font color="blue">  WABBLES implementation </font> 

In [12]:
# The following fors allow the user to automatically test different wabbles combinations
for i_phi_name in range(len(phi_names)):  
    for i_wavelet_name in range(len(wavelet_names)):   
        for i_gd_epochs in range(len(gd_epochs_options)):               
                for i_alpha in range(len(alpha_options)):  
                    for i_N1 in range(len(N1_bls_options)):               
                            for i_N3 in range(len(N3_bls_options)):     

                                # Initialize metrics
                                start_time = time.time()

                                minAcc = 100 
                                maxAcc = 0 
                                avAcc = 0

                                minFscore = 100 
                                maxFscore = 0
                                avFscore = 0

                                minPrec = 100 
                                maxPrec = 0
                                avPrec = 0

                                minRecall = 100 
                                maxRecall = 0
                                avRecall = 0

                                minSpec = 100 
                                maxSpec = 0
                                avSpec = 0

                                gradient_descent_epochs = gd_epochs_options[i_gd_epochs]
                                alpha = alpha_options[i_alpha]
                                N1_bls = N1_bls_options[i_N1]
                                N3_bls = N3_bls_options[i_N3]
                                wavelet_name = wavelet_names[i_wavelet_name]
                                phi_name = phi_names[i_phi_name]
                                ### ---

                                if dataset_simulation_type == 0: # 0 = 3-median, 1 = Real_LC
                                    csv_name = csv_path + '3median/3med_Raw_classThreshold_'+str(class_threshold)+'_wavelet_'+str(wavelet_name)+'_phiFunction_'+str(phi_name)+'_GDEpochs_'+str(gradient_descent_epochs)+'_alpha_'+str(alpha)+'_N1_'+str(N1_bls)+'_N3_'+str(N3_bls)+'_invertLCs_'+str(invert_LC)+'_runs_'+str(runs)+'.csv'
                                elif dataset_simulation_type == 1:
                                    csv_name = csv_path + 'RealLC/RealLC_Raw_classThreshold_'+str(class_threshold)+'_wavelet_'+str(wavelet_name)+'_phiFunction_'+str(phi_name)+'_GDEpochs_'+str(gradient_descent_epochs)+'_alpha_'+str(alpha)+'_N1_'+str(N1_bls)+'_N3_'+str(N3_bls)+'_invertLCs_'+str(invert_LC)+'_runs_'+str(runs)+'.csv'  
                                else:  
                                    print("Error.")
                                    sys.exit()
                                    
                                print("Start.")
                                with open(csv_name, 'w', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(["WABBLES"])
                                    writer.writerow(["Accuracy", "F-Score", "Precision", "Sensitivity", "Specificity", "TP", "TN", "FP", "FN", "Epoch time (segs)"])  

                                    for run in range(0, runs):        
                                        run_time = time.time()

                                        #if run % 5 == 0:
                                        print(color.BOLD + str(run) + "\n" + 'Raw_classThreshold_'+str(class_threshold)+'_wavelet_'+str(wavelet_name)+'_phiFunction_'+str(phi_name)+'_GDEpochs_'+str(gradient_descent_epochs)+'_alpha_'+str(alpha)+'_N1_'+str(N1_bls)+'_N3_'+str(N3_bls)+'_invertLCs_'+str(invert_LC)+'_runs_'+str(runs) + "\n\n" + color.END)
                                        
                                        # Split the data to create the train and test sets
                                        random_shuffle_state = random.randint(0, max_rand)
                                        df = pd.DataFrame(light_curves)
                                        x_train, x_test, y_train, y_test = train_test_split(df, light_curve_labels, test_size=0.4, shuffle=True, random_state=random_shuffle_state)   

                                        print("Loading the datasets.")
                                        # Load the datasets
                                        train_dataset = convert_x_y_to_array(x_train, y_train)
                                        test_dataset = convert_x_y_to_array(x_test, y_test)                        

                                        # Normalize training data
                                        #print("Normalizing the train data.")
                                        train_x = train_dataset[:, 0:train_dataset.shape[1] - 1 ];
                                        train_x = zscore(train_x, axis = 0, ddof = 1); # For each feature, mean = 0 and std = 1 
                                        replaceNan(train_x);                           # Replace "nan" with 0
                                        train_y = train_dataset[:,train_dataset.shape[1]  - 1 : train_dataset.shape[1]]; 

                                        # Normalize test data
                                        #print("Normalizing the test data.")
                                        test_x = test_dataset[:, 0:test_dataset.shape[1] - 1 ];
                                        test_x = zscore(test_x, axis = 0, ddof = 1); # For each feature, mean = 0 and std = 1 
                                        replaceNan(test_x); # Replace "nan" with 0
                                        test_y = test_dataset[:, test_dataset.shape[1]  - 1 : test_dataset.shape[1] ];
                                

                                        """ !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                                                WABBLES Execution starts here 
                                            ¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡¡
                                        """
                                        np.random.seed(seed); # set the seed for generating random numbers
                                        
                                        TrainingAccuracy, TestingAccuracy, TestingPrecision, TestingRecall, TestingSpecificity, TP, TN, FP, FN, Training_time, Testing_time, f_score, last_epoch_saved, last_acc_obtained = \
                                        wabbles(train_x,train_y,test_x,test_y,N1_bls,N3_bls, err_threshold, gradient_descent_epochs, alpha, wavelet_name, phi_name, class_threshold, writer, parameters_path, load_parameters, train_model, save_parameters, invert_LC)

                                        bls_test_acc = TestingAccuracy;
                                        bls_test_f_score = f_score;
                                        bls_train_time = Training_time;
                                        bls_test_time = Testing_time;

                                        # Set accuracy measurements
                                        minAcc, maxAcc = update_min_max(bls_test_acc, minAcc, maxAcc)                                        
                                        # Set F-score measurements
                                        minFscore, maxFscore = update_min_max(bls_test_f_score, minFscore, maxFscore)                                        
                                        # Set Precision measurements
                                        minPrec, maxPrec = update_min_max(TestingPrecision, minPrec, maxPrec)
                                        # Set Recall measurements
                                        minRecall, maxRecall = update_min_max(TestingRecall, minRecall, maxRecall)
                                        # Set Specificity measurements
                                        minSpec, maxSpec = update_min_max(TestingSpecificity, minSpec, maxSpec)

                                        avAcc += bls_test_acc
                                        avFscore += bls_test_f_score
                                        avPrec += TestingPrecision
                                        avRecall += TestingRecall
                                        avSpec += TestingSpecificity

                                        run_time = time.time() - run_time  

                                        writer.writerow([round(bls_test_acc, 2), round(bls_test_f_score, 2), round(TestingPrecision, 2),
                                                         round(TestingRecall, 2), round(TestingSpecificity, 2), round(TP, 2),
                                                         round(TN, 2), round(FP, 2), round(FN, 2), round(run_time, 2)]) 

                                        del x_train, x_test, y_train, y_test, train_dataset, test_dataset, train_x, train_y, df

                                    # Obtain averages of the metrics
                                    avAcc = avAcc/runs
                                    avFscore = avFscore/runs
                                    avPrec = avPrec/runs
                                    avRecall = avRecall/runs
                                    avSpec = avSpec/runs

                                    total_time = time.time() - start_time

                                    print(color.BOLD + str(run) + "\n" + 'Raw_classThreshold_'+str(class_threshold)+'_wavelet_'+str(wavelet_name)+'_phiFunction_'+str(phi_name)+'_GDEpochs_'+str(gradient_descent_epochs)+'_alpha_'+str(alpha)+'_N1_'+str(N1_bls)+'_N3_'+str(N3_bls)+'_invertLCs_'+str(invert_LC)+'_runs_'+str(runs) + "\n\n" + color.END)
                                        
                                    print("Average " + color.BOLD + "accuracy: " + color.END + str(round(avAcc, 2)) + "%")
                                    print("Minimum accuracy: " + str(round(minAcc, 2)) + "%")
                                    print("Maximum accuracy: " + str(round(maxAcc, 2)) + "%")
                                    print("Average " + color.BOLD + "F-score: " + color.END + str(round(avFscore, 2)) + "%")
                                    print("Minimum F-score: " + str(round(minFscore, 2)) + "%")
                                    print("Maximum F-score: " + str(round(maxFscore, 2)) + "%")
                                    print("Average " + color.BOLD + "precision: " + color.END + str(round(avPrec, 2)) + "%")
                                    print("Minimum precision: " + str(round(minPrec, 2)) + "%")
                                    print("Maximum precision: " + str(round(maxPrec, 2)) + "%")
                                    print("Average " + color.BOLD + "sensitivity: " + color.END + str(round(avRecall, 2)) + "%")
                                    print("Minimum sensitivity: " + str(round(minRecall, 2)) + "%")
                                    print("Maximum sensitivity: " + str(round(maxRecall, 2)) + "%")
                                    print("Average " + color.BOLD + "specificity: " + color.END + str(round(avSpec, 2)) + "%")
                                    print("Minimum specificity: " + str(round(minSpec, 2)) + "%")
                                    print("Maximum specificity: " + str(round(maxSpec, 2)) + "%")

                                    print("Total time: " + str(round(total_time, 2)) + " segs.\n") 

                                    writer.writerow(["Min results"])
                                    writer.writerow([round(minAcc, 2), round(minFscore, 2), round(minPrec, 2),
                                                     round(minRecall, 2), round(minSpec, 2)])
                                    writer.writerow(["Max results"])
                                    writer.writerow([round(maxAcc, 2), round(maxFscore, 2), round(maxPrec, 2),
                                                     round(maxRecall, 2), round(maxSpec, 2)])
                                    writer.writerow(["Averaged results"])
                                    writer.writerow([round(avAcc, 2), round(avFscore, 2), round(avPrec, 2),
                                                     round(avRecall, 2), round(avSpec, 2), round(total_time, 2)]) 
                                    writer.writerow(["Epoch saved", "Testing accuracy saved"])
                                    writer.writerow([last_epoch_saved, last_acc_obtained])  
                                    
                                    print("\nLast epoch that parameters were saved:", last_epoch_saved)
                                    print("Testing accuracy in the last epoch that parameters were  saved:", last_acc_obtained)
print("Done.")

Start.
0
Raw_classThreshold_0.5_wavelet_mexHat_phiFunction_tanh_GDEpochs_5_alpha_0.001_N1_3_N3_4_invertLCs_False_runs_2


Loading the datasets.
D:/Documentos/UDLAP/Tesis/Astronomía/Experiments/MRA in LCs/BLS Experiments/Wabbles/Github code/parameters/3median_
Mapping initialization is complete.
Enhancement initialization is complete.
Training the mapping and enhancement nodes.



__________________________________________________________________________
Training epoch 1 error was 838437.0117921905.
TP, TN, FP, FN, unknown 1991 1750 258 1 0
Testing Accuracy is :  93.525  %
Testing F-Score is :  93.89294977599623  %
Parameters saved to:  D:/Documentos/UDLAP/Tesis/Astronomía/Experiments/MRA in LCs/BLS Experiments/Wabbles/Github code/parameters/3median_



__________________________________________________________________________
Training epoch 2 error was 270673.7363808646.
TP, TN, FP, FN, unknown 1988 1767 241 4 0
Testing Accuracy is :  93.875  %
Testing F-Score is :  94.19568822553897  

# <font color="blue">  Unitary tests </font> 

Only run this cell if you want to execute the unitary tests


In [12]:
import os

current_dir = os.getcwd() 
current_dir = current_dir.replace("\\", "/")

parameters_test_path = current_dir + "/parameters/" 

%run -i TestWABBLES

..........C:\Users\migue\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)
.........
----------------------------------------------------------------------
Ran 19 tests in 0.041s

OK
